In [ ]:
import os

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
from google.colab import files

Mounted at /content/drive


In [ ]:
import pandas as pd
import numpy as np

In [ ]:
# Original CSV
df = pd.read_csv('/content/drive/MyDrive/BTT CARI-CONNECT /CariConnect Dataset /isbndb-caribbean-books.csv', encoding= 'latin1')

In [ ]:
df_match = df.copy()
# Remove duplicates
df_match.drop_duplicates(subset = 'title', inplace=True)
df_match.drop_duplicates(subset = 'synopsis', inplace=True)
# Remove independently published books and books missing publishers
df_match = df_match[~df_match['publisher'].str.contains("ndependent", na=False)]
df_match.dropna(subset='publisher', inplace=True)
# Remove books missing both subject and synopsis
df_match.dropna(subset=['subjects', 'synopsis'], inplace=True, how='all')
# Drop irrelevant columns
df_match.drop(['title_long', 'binding', 'pages', 'image', 'isbn', 'isbn10', 'isbn13', 'authors', 'msrp', 'edition', 'date_published'], axis=1, inplace=True)
# Filling missing data with empty strings
df_match['subjects'].fillna(value='', inplace=True)
df_match['synopsis'].fillna(value='', inplace=True)

<ipython-input-5-b43f654cb092>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_match.dropna(subset='publisher', inplace=True)
<ipython-input-5-b43f654cb092>:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_match.dropna(subset=['subjects', 'synopsis'], inplace=True, how='all')
<ipython-input-5-b43f654cb092>:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_match.drop(['title_long', 'binding', 'pages', 'image', 'isbn', 'isbn10'

In [ ]:
!pip install langdetect
!pip install deep_translator
from langdetect import detect, DetectorFactory
from langdetect.lang_detect_exception import LangDetectException
from deep_translator import GoogleTranslator

DetectorFactory.seed = 0

# Input language determinate
class LanguageDetector:
    def __init__(self, text=""):
        self.text = text

    def set_text(self, text):
        self.text = text

    def detect_language(self):
        try:
            language_code = detect(self.text)
            return language_code
        except LangDetectException:
            language_code = "en"
            return language_code

# Get the Language Preferred Publisher
class LanguagePreferredPublisher:
    def __init__(self, dataset):
        self.dataset = dataset

    def get_publishers_by_language(self, language_code):
        """
        Filters and returns a list of unique publishers that accept the given language.
        """
        # Filter dataset for rows with the specified language
        filtered_df = self.dataset[self.dataset['language'] == language_code]

        # Get unique publishers who accept this language
        publishers = filtered_df['publisher'].unique().tolist()

        if publishers:
            print(f"Publishers that accept language '{language_code}': {publishers}")
        else:
            print(f"No publishers found for language '{language_code}'.")

        return publishers

def translate_to_english(text):
    try:
        translated = GoogleTranslator(source='auto', target='en').translate(text)
        return translated
    except:
        return text

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 kB 6.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for langdetect: filename=langdetect-1.0.9-py3-none-any.whl size=993222 sha256=bc88db25dacd6faa86fad6b574871a0248b01f40f0d1a9adc452f8987bdbd7eb
  Stored in directory: /root/.cache/pip/wheels/95/03/7d/59ea870c70ce4e5a370638b5462a7711ab78fba2f655d05106
Successfully built langdetect
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.3/42.3 kB 1.3 MB/s eta 0:00:00


In [ ]:
# Detect languages and translate synopsis to English
detector = LanguageDetector()
for idx, row in df_match.iterrows():
  detector.set_text(row['synopsis'])
  detected_language = detector.detect_language()
  df_match.at[idx, 'language'] = detected_language
  df_match.at[idx, 'synopsis'] = translate_to_english(row['synopsis'])
  #publisher_filter = LanguagePreferredPublisher(df)
  #publishers_for_french = publisher_filter.get_publishers_by_language('fr')

In [ ]:
# Model dataframe
df_model = df_match.copy()
df_model.drop(['publisher', 'language'], axis=1, inplace=True)

In [ ]:
# Text Preprocessing
import string
import re
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
nltk.download('punkt')
nltk.download('punkt_tab')
nltk.download('stopwords')
nltk.download('wordnet')
from nltk.stem.snowball import SnowballStemmer
stemmer = SnowballStemmer("english")
import sys
!{sys.executable} -m pip install contractions
import contractions

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 289.9/289.9 kB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.7/110.7 kB 7.6 MB/s eta 0:00:00


In [ ]:
def preprocess(s):
    s=str(s)
    s = s.lower() # lowercase
    s = contractions.fix(s) # expand contractions
    s = re.sub(r'\\n', ' ', s) # remove \n
    s = re.sub(r'http\S+', '', s) # remove url
    s = re.sub(r'<.*?>', '', s) # remove html
    s = re.sub(r'\d+', '', s) # remove numbers
    s = re.sub(r'[^\w\s]', ' ', s) # remove punctuation and special characters
    s = word_tokenize(s) # tokenize
    s = [w for w in s if w not in set(stopwords.words('english'))] # stop words
    s = [stemmer.stem(w) for w in s] # stemming
    return " ".join(s) # white spaces

In [ ]:
df_model=df_model.map(lambda s:preprocess(s)).copy()

In [ ]:
# TF IDF
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf = TfidfVectorizer(ngram_range=(1,2), min_df=5, max_df=.99)

In [ ]:
# merged TF IDF
df_model['merged'] = df_model['title'] + ' ' + df_model['subjects'] + ' ' + df_model['synopsis']
tfidf.fit(df_model['merged'])

TfidfVectorizer(max_df=0.99, min_df=5, ngram_range=(1, 2))

In [ ]:
# column weights
title_weight=.05
subject_weight=.8
synopsis_weight=.15

In [ ]:
# Apply TF-IDF to each column
def tfidfer(df_col):
  matrix = tfidf.transform(df_col)
  return pd.DataFrame(matrix.toarray(), columns=tfidf.get_feature_names_out(), index = df_model.index)
title_tfidf = title_weight*tfidfer(df_model['title'])
subject_tfidf = subject_weight*tfidfer(df_model['subjects'])
synopsis_tfidf = synopsis_weight*tfidfer(df_model['synopsis'])
tfidf_df = pd.concat([title_tfidf,subject_tfidf,synopsis_tfidf], axis=1)

In [ ]:
vector_df = tfidf_df.copy()

In [ ]:
# Scaling
from sklearn.preprocessing import MaxAbsScaler
max_scaler = MaxAbsScaler()
max_df = max_scaler.fit(vector_df)
max_df = pd.DataFrame(max_scaler.transform(vector_df), columns=vector_df.columns, index=vector_df.index)

In [ ]:
scaled_df = max_df.copy()

In [ ]:
# SVD dimension reduction
from sklearn.decomposition import TruncatedSVD
svd = TruncatedSVD(n_components=50, algorithm='arpack')
svd.fit(scaled_df)

TruncatedSVD(algorithm='arpack', n_components=50)

In [ ]:
svd_df = pd.DataFrame(svd.transform(scaled_df), index=vector_df.index)

In [ ]:
reduced_df=svd_df.copy()

In [ ]:
# Birch clustering
from sklearn.cluster import Birch

In [ ]:
birch_model = Birch(threshold=0.25, n_clusters=75)

In [ ]:
cmodel = birch_model

In [ ]:
fit = cmodel.fit(reduced_df)
labels = fit.labels_
labels

array([ 2, 68, 15, ..., 67, 16, 56])

In [ ]:
df_model['label'] = labels # get labels

In [ ]:
predictions = cmodel.predict(reduced_df)

In [ ]:
# Clean and vectorize input
title_input = "Anansi"
subject_input = "Akan Folklore"
synopsis_input = "nansi or Ananse (/əˈnɑːnsi/ ə-NAHN-see; literally translates to spider) is an Akan folktale character associated with stories, wisdom, knowledge, and trickery, most commonly depicted as a spider, in Akan folklore.[1] Taking the role of a trickster, he is also one of the most important characters of West African, African American and West Indian folklore. Originating in Ghana, these spider tales were transmitted to the Caribbean by way of the transatlantic slave trade.[2]  Anansi is best known for his ability to outsmart and triumph over more powerful opponents through his use of cunning, creativity and wit.[3] Despite taking on a trickster role, Anansi often takes centre stage in stories and is commonly portrayed as both the protagonist and antagonist."
detector.set_text(synopsis_input)
detect_input = detector.detect_language()
synopsis_input = translate_to_english(synopsis_input)
lang_input = detect_input # detect language, or selected language

In [ ]:
def preprocess_transform(s):
  preprocessed = preprocess(s)
  transformed = tfidf.transform([preprocessed]).toarray()
  return transformed

In [ ]:
clean_title_input = title_weight*preprocess_transform(title_input)
clean_subject_input = subject_weight*preprocess_transform(subject_input)
clean_synopsis_input = synopsis_weight*preprocess_transform(synopsis_input)
input_df = pd.DataFrame(np.concatenate([clean_title_input, clean_subject_input, clean_synopsis_input], axis=1))
input_df

,0,1,2,3,4,5,6,7,8,9,...,38108,38109,38110,38111,38112,38113,38114,38115,38116,38117
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
# scale input
input_df = max_scaler.transform(input_df)

/usr/local/lib/python3.10/dist-packages/sklearn/base.py:493: UserWarning: X does not have valid feature names, but MaxAbsScaler was fitted with feature names
  warnings.warn(


In [ ]:
# reduce dimensions of input
svd_input = svd.transform(input_df)

/usr/local/lib/python3.10/dist-packages/sklearn/base.py:493: UserWarning: X does not have valid feature names, but TruncatedSVD was fitted with feature names
  warnings.warn(


In [ ]:
red_input = svd_input

In [ ]:
# predict test input
input_pred = cmodel.predict(red_input)[0]

In [ ]:
# vector subset where label matches
subset = df_model[df_model['label'] == input_pred]

In [ ]:
# get subset
sim_df = reduced_df.copy()
sim_subset = sim_df.loc[subset.index]

In [ ]:
# calculate cosine similarity within that cluster
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.metrics.pairwise import euclidean_distances
simscore = cosine_similarity
similarity = simscore(red_input, sim_subset).T #input
similarity

array([[0.45655723],
       [0.46806339],
       [0.67626253],
       [0.52413074],
       [0.50361636],
       [0.31344924],
       [0.2251212 ],
       [0.31207694],
       [0.53817905],
       [0.44186113],
       [0.69645892],
       [0.09071123],
       [0.19627309],
       [0.52670208],
       [0.54847058],
       [0.47807647],
       [0.47213743],
       [0.14700276],
       [0.25144722],
       [0.18334766],
       [0.64296533],
       [0.24979952],
       [0.34967418],
       [0.46522348],
       [0.59451042],
       [0.39475002],
       [0.53437828],
       [0.40368373],
       [0.10223216],
       [0.11776082],
       [0.13670262],
       [0.21458065],
       [0.34085517],
       [0.59927985],
       [0.67180402],
       [0.68188886],
       [0.56171959],
       [0.56135217],
       [0.27751134],
       [0.3983452 ],
       [0.12325798],
       [0.20473526],
       [0.38990069],
       [0.52290792],
       [0.08266409],
       [0.13170416],
       [0.85579988],
       [0.624

In [ ]:
# append similarity to dataframe
subset['similarity'] = similarity

<ipython-input-63-bbfe0a642cef>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  subset['similarity'] = similarity


In [ ]:
subset.loc[df_match['language'] == lang_input, 'similarity']+=1 # prioritize same language

In [ ]:
# top 5 similar
top_subset = subset.sort_values(by='similarity', ascending=False).head(5)
top_subset

,title,subjects,synopsis,merged,label,similarity
1183,string pearl caribbean travel beyond,,product descriptionmeet caribbean insight humo...,string pearl caribbean travel beyond product ...,40,1.867122
1280,adult educ caribbean turn centuri,,burnout common among doctor west one might ass...,adult educ caribbean turn centuri burnout com...,40,1.858066
879,caribbean flame,romanc,damon alexand resist ramona king although unaw...,caribbean flame romanc damon alexand resist ra...,40,1.855800
1449,caribbean raj,,book collect poetri bring life work woman fit ...,caribbean raj book collect poetri bring life ...,40,1.852289
3730,caribbean dream,,review vast entertain work set fictiti caribbe...,caribbean dream review vast entertain work se...,40,1.839351


In [ ]:
top_index = top_subset.index
top_index

Index([1183, 1280, 879, 1449, 3730], dtype='int64')

In [ ]:
# get information of top 5
top_titles = []
top_subjects = []
top_synopsis = []
top_publishers = []
top_languages = []

In [ ]:
def book_info(index):
  title = df_match.loc[index]['title']
  subjects = df_match.loc[index]['subjects']
  synopsis = df_match.loc[index]['synopsis']
  publisher = df_match.loc[index]['publisher']
  language = df_match.loc[index]['language']
  return title, subjects, synopsis, publisher, language

In [ ]:
for i in range(len(top_index)):
  top_titles.append(book_info(top_index[i])[0])
  top_subjects.append(book_info(top_index[i])[1])
  top_synopsis.append(book_info(top_index[i])[2])
  top_publishers.append(book_info(top_index[i])[3])
  top_languages.append(book_info(top_index[i])[4])
  print(book_info(top_index[i])[0])

A String of Pearls: Caribbean Travels and Beyond
Adult education in the Caribbean at the turn of the century
Caribbean Flame
The Caribbean Raj
Caribbean Dreams


In [ ]:
# GROQ API to prompt LLAMA
!pip install groq

In [ ]:
%env GROQ_API_KEY=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

env: GROQ_API_KEY=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


In [ ]:
from groq import Groq

In [ ]:
client = Groq(
    api_key=os.environ.get("GROQ_API_KEY"),
)

In [ ]:
chat_completion = client.chat.completions.create(
    #
    # Required parameters
    #
    messages=[
        # Set an optional system message. This sets the behavior of the
        # assistant and can be used to provide specific instructions for
        # how it should behave throughout the conversation.
        {
            "role": "system",
            "content": f"You are a knowledgable and straightforward assistant with experience in literature and literary analysis, and knowledge of the publishing industry. You are explaining to authors which books are most similar to theirs using criteria such as themes, plot, character, setting, and tone. Your responses are concise and academic, strictly providing lists and explanations. Structure the response in a list, where each entry is formatted, with elaboration as necessary. (Do not mention the books' alphabetical labels A B C D E) : \n (Rank number). (Publisher) \n Published in (Language)\n Explanation: (Publisher) published (Title), a (Subject) book that is similar to your book because (Explanation, 3 sentence justification of the book similarities). Therefore, your book appeals to (Publisher)'s sector of the market."
        },
        # Set a user message for the assistant to respond to.
        {
            "role": "user",
            "content": f"Using the input book information as a reference, compare and rank the similarity of books A, B, C, D, and E to the reference. Use literary analysis to evaluate each book based on thematic alignment, such as central conflict, character focus, and setting, and stylistic features such as narrative tone and diction. \n Provide a list of the respective publishers of the books in descending order of book similarity to the reference book with a 3 sentence justification of the book similarities and ranking, citing specific aspects of the book information and synopsis that supports your assessment. Structure the response in a list, where each entry is formatted, with elaboration as necessary. (Do not mention the books' alphabetical labels A B C D E) : \n (Rank number). (Publisher) \n Published in (Language)\n Explanation: (Publisher) published (Title), a (Subject) book that is similar to your book because (Explanation, 3 sentence justification of the book similarities). Therefore, your book appeals to (Publisher)'s sector of the market. \n \n Book Reference: \n Title: {title_input}. \n Subject: {subject_input} \n Synopsis: {synopsis_input} \n \n Book A: \n Title: {top_titles[0]} \n Subject: {top_subjects[0]} \n Synopsis: {top_synopsis[0]} \n Publisher: {top_publishers[0]} \n Language: {top_languages[0]} \n \n Book B: \n Title: {top_titles[1]} \n Subject: {top_subjects[1]} \n Synopsis: {top_synopsis[1]} \n Publisher: {top_publishers[1]} \n Language: {top_languages[1]} \n \n Book C: \n Title: {top_titles[2]} \n Subject: {top_subjects[2]} \n Synopsis: {top_synopsis[2]} \n Publisher: {top_publishers[2]} \n Language: {top_languages[2]} \n \n Book D: \n Title: {top_titles[3]} \n Subject: {top_subjects[3]} \n Synopsis: {top_synopsis[3]} \n Publisher: {top_publishers[3]} \n Language: {top_languages[3]} \n \n Book E: \n Title: {top_titles[4]} \n Subject: {top_subjects[4]} \n Synopsis: {top_synopsis[4]} \n Publisher: {top_publishers[4]} \n Language: {top_languages[4]}"

        }

    ],

    # The language model which will generate the completion.
    model= "llama3-70b-8192",#"llama3-70b-8192", "llama3-8b-8192", "llama-3.1-70b-versatile", "llama-3.1-8b-instant"

    #
    # Optional parameters
    #

    # Controls randomness: lowering results in less random completions.
    # As the temperature approaches zero, the model will become deterministic
    # and repetitive.
    temperature=0.3,

    # The maximum number of tokens to generate. Requests can use up to
    # 32,768 tokens shared between prompt and completion.
    max_tokens=8000,

    # Controls diversity via nucleus sampling: 0.5 means half of all
    # likelihood-weighted options are considered.
    top_p=1,

    # A stop sequence is a predefined or user-specified text string that
    # signals an AI to stop generating content, ensuring its responses
    # remain focused and concise. Examples include punctuation marks and
    # markers like "[end]".
    stop=None,

    # If set, partial message deltas will be sent.
    stream=False,
)

In [ ]:
# Print the completion returned by the LLM.
output = chat_completion.choices[0].message.content

In [ ]:
output = GoogleTranslator(source='auto', target=lang_input).translate(output)

In [ ]:
print(output)

Here is the list of publishers in descending order of book similarity to the reference book "Anansi" with a 3-sentence justification of the book similarities:

1. Ian Randle Publishers, Jamaica
Published in: en
Explanation: Ian Randle Publishers, Jamaica published "The Caribbean Raj", a poetry collection that is similar to "Anansi" because it explores the cultural heritage and identity of the Caribbean region, much like Anansi's role in Akan folklore. The book's themes of colonialism, identity, and cultural expression align with the trickster character's ability to outsmart and triumph over more powerful opponents. Therefore, your book appeals to Ian Randle Publishers' sector of the market.

2. Olympia Publishers
Published in: en
Explanation: Olympia Publishers published "A String of Pearls: Caribbean Travels and Beyond", a travelogue that is similar to "Anansi" because it explores the cultural diversity and richness of the Caribbean region, much like Anansi's role in Akan folklore. Th